# Connection au Drive

In [ ]:
import pandas as pd
import os
from google.colab import drive
drive.mount('/content/drive')
os.getcwd()  # Lecture et résolution depuis un fichier


# Chemin vers le dossier ajouté à Mon Drive
chemin_dossier = '/content/drive/My Drive/PIP2025_G7_Elections/Données propres/Legislative/mesdatapropre'

# Naviguer vers le dossier
os.chdir(chemin_dossier)

# Lister les fichiers dans le dossier
print("Fichiers dans le dossier :", os.listdir())

## Graphique interactif Dash sur 2024 seulement

In [ ]:
! pip install plotly
! pip install dash
! pip install streamlit

import pandas as pd
import dash
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash.dependencies as dd

# === 1. CHARGEMENT ET PRÉPARATION DES DONNÉES ===
data = pd.read_csv("Circo.leg.T2.2024.csv", delimiter=';', encoding='utf-8')

# Colonnes (adaptées à votre jeu de données)
colonnes_partis = ['Nuance candidat 1', 'Nuance candidat 2', 'Nuance candidat 3', 'Nuance candidat 4']
colonnes_elus = ['Elu 1', 'Elu 2', 'Elu 3', 'Elu 4']
colonnes_voix = ['Voix 1', 'Voix 2', 'Voix 3', 'Voix 4']
colonnes_sexe = ['Sexe candidat 1', 'Sexe candidat 2', 'Sexe candidat 3', 'Sexe candidat 4']

# Construit un DataFrame long avec 1 ligne par candidat
long_df = pd.DataFrame({
    "Nuance": pd.concat([data[col] for col in colonnes_partis], ignore_index=True),
    "Voix": pd.concat([data[col] for col in colonnes_voix], ignore_index=True),
    "Elu": pd.concat([data[col].apply(lambda x: 1 if str(x).strip().lower() == 'élu' else 0)
                      for col in colonnes_elus], ignore_index=True),
    "Sexe": pd.concat([data[col] for col in colonnes_sexe], ignore_index=True)
})

# Nettoyage de la colonne Sexe : enlever NaN, espaces, etc.
long_df['Sexe'] = long_df['Sexe'].fillna('').astype(str).str.strip().str.upper()
long_df = long_df[long_df['Sexe'].isin(['MASCULIN', 'FEMININ'])]  # Filtre si besoin

# Liste des sexes uniques (pour la checklist)
unique_sexes = long_df['Sexe'].unique().tolist()

# === 2. CRÉATION DE L'APPLICATION DASH ===
app = dash.Dash(__name__)

app.layout = html.Div([
    # Checklist pour filtrer par sexe
    dcc.Checklist(
        id='sex-checklist',
        options=[{"label": s, "value": s} for s in unique_sexes],
        value=unique_sexes,  # par défaut, tout coché
        labelStyle={'display': 'block'}
    ),

    # Graphique (bar chart)
    dcc.Graph(id='my-graph')
])

# === 3. CALLBACK POUR METTRE À JOUR LE GRAPHIQUE SELON LE SEXE ===
@app.callback(
    dd.Output('my-graph', 'figure'),
    dd.Input('sex-checklist', 'value')
)
def update_graph(selected_sexes):
    """
    Filtre les données sur les sexes sélectionnés et génère un bar chart
    comparant le total de Voix et le total d'Elu par Nuance.
    """
    if not selected_sexes:
        # Si rien n'est coché, on peut soit retourner un graph vide, soit tout masquer
        return go.Figure()

    # Filtrage du DataFrame sur les sexes choisis
    filtered_df = long_df[long_df['Sexe'].isin(selected_sexes)]

    # Agrégation par Nuance : somme de Voix et somme d'Elu
    agg_df = filtered_df.groupby("Nuance", dropna=False).agg({
        "Voix": "sum",
        "Elu": "sum"
    }).reset_index()

    # Tri par Voix (optionnel)
    agg_df = agg_df.sort_values(by="Voix", ascending=False)

    # Construction du bar chart
    fig = go.Figure()

    # Barres pour les Voix
    fig.add_trace(go.Bar(
        x=agg_df['Nuance'],
        y=agg_df['Voix'],
        name='Voix',
        marker_color='blue',
        text=agg_df['Voix'],
        textposition='outside'
    ))

    # Barres pour les Élus
    fig.add_trace(go.Bar(
        x=agg_df['Nuance'],
        y=agg_df['Elu'],
        name='Élus',
        marker_color='orange',
        text=agg_df['Elu'],
        textposition='outside'
    ))

    # Mise en forme générale
    fig.update_layout(
        title="Nombre de Voix et d'Élus par Parti (filtrés par Sexe)",
        xaxis_title="Nuance (Parti)",
        yaxis_title="Nombre",
        barmode='group',
        bargap=0.15,
        uniformtext_minsize=8,
        uniformtext_mode='hide'
    )

    return fig


# === 4. LANCEMENT DU SERVEUR ===
if __name__ == "__main__":
    app.run_server(debug=True)



<IPython.core.display.Javascript object>

# Ensembler les differents  fichier des votes

In [ ]:
import pandas as pd
import numpy as np

def load_election_data(filename: str, year: int) -> pd.DataFrame:
    """
    Lit le fichier CSV 'filename' et renvoie un DataFrame "long"
    avec les colonnes : [Année, Nuance, Voix, Elu, Sexe].

    Cette fonction gère plusieurs formats de colonnes selon l'année :
      - 2002, 2007, 2012 (pas de 'Elu' explicite)
      - 2017 (colonnes "Nuance", "Nuance.1", ..., "Sièges", "Sièges.1", etc. contenant "Elu")
      - 2022, 2024 (colonnes "Nuance1", "Voix1", "Elu1", etc.)

    year = par ex. 2002, 2007, 2012, 2017, 2022, 2024, etc.
    """

    # Lecture du CSV en texte
    df = pd.read_csv(filename, delimiter=';', encoding='utf-8', dtype=str)

    # ------------------------------------------------------------------------------
    # 1) CAS 2022 / 2024 : on a par exemple "Nuance1", "Voix1", "Elu1", "Sexe1" ...
    # ------------------------------------------------------------------------------
    if year >= 2022:
        # On suppose jusqu’à 4 candidats maxi (cf. vos exemples T2)
        nb_candidats = 4
        records = []
        for i in range(1, nb_candidats + 1):
            col_nuance = f"Nuance{i}"
            col_voix   = f"Voix{i}"
            col_elu    = f"Elu{i}"
            col_sexe   = f"Sexe{i}"

            # Vérifions l'existence de ces colonnes
            if (col_nuance in df.columns) and (col_voix in df.columns) and (col_sexe in df.columns):
                nuance_vals = df[col_nuance].fillna('').str.strip()
                voix_vals   = pd.to_numeric(df[col_voix].fillna('0'), errors='coerce').fillna(0)
                # "Elu" = 1 si la colonne contient "Elu", sinon 0
                if col_elu in df.columns:
                    elu_vals = df[col_elu].fillna('').str.lower().str.strip().apply(
                        lambda x: 1 if 'elu' in x else 0
                    )
                else:
                    # S'il n'y a pas de colonne EluX, on met 0
                    elu_vals = pd.Series([0]*len(df))

                # Sexe (M/F), on normalise si besoin
                sexe_vals = df[col_sexe].fillna('').str.strip().str.upper()

                tmp_df = pd.DataFrame({
                    "Année":  [year]*len(df),
                    "Nuance": nuance_vals,
                    "Voix":   voix_vals,
                    "Elu":    elu_vals,
                    "Sexe":   sexe_vals
                })
                records.append(tmp_df)

        if records:
            final_df = pd.concat(records, ignore_index=True)
        else:
            final_df = pd.DataFrame(columns=["Année","Nuance","Voix","Elu","Sexe"])
        # Nettoyage (retirer les nuances vides, etc.)
        final_df = final_df[final_df["Nuance"] != ""]

        return final_df

    # ------------------------------------------------------------------------------
    # 2) CAS 2017 : colonnes "Nuance", "Nuance.1", "Nuance.2"... + "Sièges", "Sièges.1"...
    # ------------------------------------------------------------------------------
    elif year == 2017:
        # On part du principe qu'il peut y avoir jusqu'à 9 candidats (Nuance, Nuance.1, .2, etc.)
        # Adaptez si vous savez qu'il n'y en a que 2 ou 3
        max_candidates = 10
        records = []

        for i in range(max_candidates):
            # Construction du suffixe vide pour i=0, sinon .i
            suffix = "" if i == 0 else f".{i}"

            col_nuance = f"Nuance{suffix}"
            col_voix   = f"Voix{suffix}"
            col_sexe   = f"Sexe{suffix}"
            col_sieges = f"Sièges{suffix}"  # contient "Elu" ?

            # Vérifions l'existence
            if (col_nuance in df.columns) and (col_voix in df.columns) and (col_sexe in df.columns):
                nuance_vals = df[col_nuance].fillna('').str.strip()
                voix_vals   = pd.to_numeric(df[col_voix].fillna('0'), errors='coerce').fillna(0)
                sexe_vals   = df[col_sexe].fillna('').str.strip().str.upper()

                # Elu : si la colonne Sièges{suffix} existe et contient "Elu"
                if col_sieges in df.columns:
                    elu_series = df[col_sieges].fillna('').str.lower().str.strip().apply(
                        lambda x: 1 if 'elu' in x else 0
                    )
                else:
                    elu_series = pd.Series([0]*len(df))

                tmp_df = pd.DataFrame({
                    "Année":  [year]*len(df),
                    "Nuance": nuance_vals,
                    "Voix":   voix_vals,
                    "Elu":    elu_series,
                    "Sexe":   sexe_vals
                })
                records.append(tmp_df)

        # Concaténer
        if records:
            final_df = pd.concat(records, ignore_index=True)
        else:
            final_df = pd.DataFrame(columns=["Année","Nuance","Voix","Elu","Sexe"])

        final_df = final_df[final_df["Nuance"] != ""]
        return final_df

    # ------------------------------------------------------------------------------
    # 3) CAS 2002 / 2007 / 2012 : colonnes "Nuance", "Nuance.1", "Nuance.2", ...
    #    A priori pas de mention "Elu", ni "Sièges". On mettra Elu=0 par défaut.
    # ------------------------------------------------------------------------------
    else:
        # Pareil, on suppose x candidats => colonnes "Nuance", "Nuance.1", ...
        # + "Voix", "Voix.1", ... + "Sexe", "Sexe.1", ...
        max_candidates = 10
        records = []

        for i in range(max_candidates):
            suffix = "" if i == 0 else f".{i}"
            col_nuance = f"Nuance{suffix}"
            col_voix   = f"Voix{suffix}"
            col_sexe   = f"Sexe{suffix}"

            # Sur 2002/2007/2012, d'après les exemples :
            # on a "Sexe", "Nom", "Prénom", "Nuance", "Voix" (pas de .1 ?).
            # Pour le 2e candidat => "Sexe.1", "Nom.1", ...
            # => D'où ce suffix.
            # Vérifions l'existence
            if col_nuance in df.columns and col_voix in df.columns and col_sexe in df.columns:
                nuance_vals = df[col_nuance].fillna('').str.strip()
                voix_vals   = pd.to_numeric(df[col_voix].fillna('0'), errors='coerce').fillna(0)
                sexe_vals   = df[col_sexe].fillna('').str.strip().str.upper()

                # Sur ces années, pas de mention explicite "Elu",
                # on met Elu=0 (ou vous pouvez calculer autrement).
                elu_vals = pd.Series([0]*len(df))

                tmp_df = pd.DataFrame({
                    "Année":  [year]*len(df),
                    "Nuance": nuance_vals,
                    "Voix":   voix_vals,
                    "Elu":    elu_vals,
                    "Sexe":   sexe_vals
                })
                records.append(tmp_df)

            # Si on n'a pas ce pattern, on peut tenter un fallback ?
            # (Par exemple, parfois la colonne s'appelle "Nuance1"?
            # Mais comme dans vos extraits 2002..2012, c'est "Nuance" + un suffix .1, .2, etc.)

        if records:
            final_df = pd.concat(records, ignore_index=True)
        else:
            # Si vraiment pas trouvé, on renvoie un DF vide
            final_df = pd.DataFrame(columns=["Année","Nuance","Voix","Elu","Sexe"])

        final_df = final_df[final_df["Nuance"] != ""]
        return final_df



In [ ]:
years_files = {
    2002: "Circo.leg.T2.2002.csv",
    2007: "Circo.leg.T2.2007.csv",
    2012: "Circo.leg.T2.2012.csv",
    2017: "Circo.leg.T2.2017.csv",
    2022: "Circo.leg.T2.2022.csv",
    2024: "Circo.leg.T2.2024.csv"
}

list_dfs = []
for y, f in years_files.items():
    df_y = load_election_data(f, y)
    list_dfs.append(df_y)

# DataFrame global
df_all = pd.concat(list_dfs, ignore_index=True)

# Eventuellement, nettoyer la colonne Sexe => MASCULIN / FEMININ
df_all["Sexe"] = df_all["Sexe"].replace({
    "M": "MASCULIN", "F": "FEMININ",
    # etc. si besoin
})
df_all["Sexe"] = df_all["Sexe"].where(df_all["Sexe"].isin(["MASCULIN","FEMININ"]), "INDETERMINE")

print(df_all)

# Filtrer les données pour l'année 2024
df_2024 = df_all[df_all['Année'] == 2022]

# Calculer le nombre total de voix par orientation politique
voix_par_orientation_2024 = df_2024.groupby('Nuance')['Voix'].sum().sort_values(ascending=False)

voix_par_orientation_2024

      Année Nuance     Voix  Elu      Sexe
0      2002    SOC  19817.0    0  MASCULIN
1      2002    SOC  15891.0    0   FEMININ
2      2002    PRG  16094.0    0  MASCULIN
3      2002    SOC  19358.0    0  MASCULIN
4      2002    SOC  22010.0    0  MASCULIN
...     ...    ...      ...  ...       ...
6516   2024     RN  16516.0    1   FEMININ
6517   2024     RN  15472.0    0  MASCULIN
6518   2024     RN  16564.0    0  MASCULIN
6519   2024     LR  11903.0    0   FEMININ
6520   2024     LR  17083.0    1  MASCULIN

[6521 rows x 5 columns]


,Voix
Nuance,
ENS,8002468.0
NUP,6556229.0
RN,3589413.0
LR,1447838.0
DVG,443283.0
REG,264779.0
DVD,231071.0
DVC,99145.0
UDI,64444.0


# Graphe Interactive avec les votes

In [ ]:
! pip install dash
import dash
from dash import dcc, html
import dash.dependencies as dd
import plotly.graph_objs as go

app = dash.Dash(__name__)

# === df_all contient toutes les années ===
# (issu du code ci-dessus)
# Colonnes: [Année, Nuance, Voix, Elu, Sexe]

# Liste des années uniques
available_years = sorted(df_all["Année"].unique())

# Liste des sexes disponibles (souvent "MASCULIN", "FEMININ")
available_sexes = sorted(s for s in df_all["Sexe"].unique() if s != "INDETERMINE")

app.layout = html.Div([
    # Dropdown pour choisir l'année
    html.Label("Choisir l'année"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{"label": str(y), "value": y} for y in available_years],
        value=2022,  # année sélectionnée par défaut
        clearable=False
    ),

    # Checklist pour le sexe
    html.Label("Filtrer par sexe"),
    dcc.Checklist(
        id='sex-checklist',
        options=[{"label": s, "value": s} for s in available_sexes],
        value=available_sexes,  # tous cochés par défaut
        labelStyle={'display': 'block'}
    ),

    # Graphique
    dcc.Graph(id='voix-elus-graph')
])

@app.callback(
    dd.Output('voix-elus-graph', 'figure'),
    [
        dd.Input('year-dropdown', 'value'),
        dd.Input('sex-checklist', 'value')
    ]
)
def update_chart(selected_year, selected_sexes):
    """
    Filtre sur l'année (selected_year) et le(s) sexe(s) (selected_sexes),
    puis agrège (Nuance -> sum(Voix), sum(Elu)) et construit le bar chart.
    """
    # 1) Filtrage
    filtered = df_all[df_all["Année"] == selected_year]
    if selected_sexes:
        filtered = filtered[filtered["Sexe"].isin(selected_sexes)]
    else:
        # si rien n'est coché, renvoyer un graph vide
        return go.Figure()

    # 2) Agrégation par Nuance
    grouped = filtered.groupby("Nuance").agg({
        "Voix": "sum",
        "Elu": "sum"
    }).reset_index()

    # Tri par Voix
    grouped = grouped.sort_values("Voix", ascending=False)

    # 3) Construction du bar chart
    fig = go.Figure()

    # Bar Voix
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Voix"],
        name="Voix",
        marker_color='blue',
        text=grouped["Voix"],
        textposition='outside'
    ))

    # Bar Élus
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Elu"],
        name="Élus",
        marker_color='orange',
        text=grouped["Elu"],
        textposition='outside'
    ))

    fig.update_layout(
        title=f"Élections {selected_year} : Voix et Élus par Nuance",
        xaxis_title="Nuance (Parti)",
        yaxis_title="Nombre total",
        barmode='group',
        bargap=0.15
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True)



<IPython.core.display.Javascript object>

# Graphe Interactive sur l'audio visuel

In [ ]:
import dash
from dash import dcc, html
import dash.dependencies as dd
import plotly.graph_objs as go
import pandas as pd

# Charger les données des chaînes transformées
df_chaine = pd.read_csv('ina-barometre-jt-tv-donnee-mois-theme-2000-2020.csv', delimiter=';')
df_chaine.columns = ['Année', 'Mois', 'Chaîne', 'PartdAudience', 'Thématique', 'Nb_Thème', 'Nb_temps']
df_chaine['Année'] = df_chaine['Année'].astype(int)
df_chaine['Nb_Thème'] = pd.to_numeric(df_chaine['Nb_Thème'], errors='coerce')
df_chaine['Nb_temps'] = pd.to_numeric(df_chaine['Nb_temps'], errors='coerce')

# Regrouper les données des chaînes par Année, Chaîne et Thématique
df_chaine_transformed = df_chaine.groupby(['Année', 'Chaîne', 'Thématique'], as_index=False).agg({
    'Nb_Thème': 'sum',
    'Nb_temps': 'sum'
})


# Application Dash
app = dash.Dash(__name__)

# Années disponibles (communes aux deux datasets)
available_years = sorted(set(df_all['Année']).union(df_chaine_transformed['Année']))
relevant_years = [2002, 2007, 2012, 2017, 2022, 2024]

# Chaînes disponibles
available_channels = df_chaine_transformed['Chaîne'].unique()

# Thématiques disponibles
available_thematics = df_chaine_transformed['Thématique'].unique()


# Application Dash
app = dash.Dash(__name__)

# Années disponibles (communes aux deux datasets)
relevant_years = [2002, 2007, 2012, 2017, 2022, 2024]

# Chaînes disponibles
available_channels = df_chaine_transformed['Chaîne'].unique()

# Thématiques disponibles
available_thematics = df_chaine_transformed['Thématique'].unique()

# Sexes disponibles
available_sexes = df_all['Sexe'].unique()

app.layout = html.Div([
    # Dropdown pour choisir l'année
    html.Label("Choisir l'année"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{"label": str(y), "value": y} for y in relevant_years],
        value=2022,
        clearable=False
    ),

    # Radio boutons pour filtrer par sexe
    html.Label("Filtrer par sexe :"),
    dcc.Checklist(
        id='sex-checklist',
        options=[{"label": s, "value": s} for s in available_sexes],
        value=available_sexes,
        labelStyle={'display': 'inline-block'}
    ),

    # Graphique des élections
    dcc.Graph(id='election-graph'),

    # Radio boutons pour les chaînes télévisées
    html.Label("Choisir la chaîne télévisée :"),
    dcc.RadioItems(
        id='channel-radio',
        options=[{'label': c, 'value': c} for c in available_channels],
        value=available_channels[0],  # Chaîne par défaut
        labelStyle={'display': 'inline-block'}
    ),

    # Checklist pour les thématiques
    html.Label("Choisir les thématiques :"),
    dcc.Checklist(
        id='thematic-checklist',
        options=[{'label': t, 'value': t} for t in available_thematics],
        value=[available_thematics[0]],  # Thématique par défaut
        labelStyle={'display': 'inline-block'}
    ),

    # Graphique des chaînes télévisées
    dcc.Graph(id='channel-graph'),

    # Bouton radio pour choisir entre Nb_Thème et Nb_temps
    html.Label("Choisir la métrique pour les chaînes télévisées :"),
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Nombre de Thèmes', 'value': 'Nb_Thème'},
            {'label': 'Temps', 'value': 'Nb_temps'}
        ],
        value='Nb_Thème',  # Par défaut, on choisit le nombre de thèmes
        labelStyle={'display': 'inline-block'}
    )
])

# Callback pour le graphique des élections
@app.callback(
    dd.Output('election-graph', 'figure'),
    [
        dd.Input('year-dropdown', 'value'),
        dd.Input('sex-checklist', 'value')
    ]
)
def update_election_graph(selected_year, selected_sexes):
    filtered = df_all[(df_all['Année'] == selected_year) & (df_all['Sexe'].isin(selected_sexes))]
    grouped = filtered.groupby("Nuance").agg({"Voix": "sum", "Elu": "sum"}).reset_index()
    grouped = grouped.sort_values("Voix", ascending=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Voix"],
        name="Voix",
        marker_color='blue',
        text=grouped["Voix"],
        textposition='outside'
    ))
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Elu"],
        name="Élus",
        marker_color='orange',
        text=grouped["Elu"],
        textposition='outside'
    ))
    fig.update_layout(
        title=f"Élections {selected_year} : Voix et Élus par Nuance",
        xaxis_title="Nuance (Parti)",
        yaxis_title="Nombre total",
        barmode='group',
        bargap=0.15
    )
    return fig

# Callback pour le graphique des chaînes télévisées
@app.callback(
    dd.Output('channel-graph', 'figure'),
    [
        dd.Input('year-dropdown', 'value'),
        dd.Input('channel-radio', 'value'),
        dd.Input('thematic-checklist', 'value'),
        dd.Input('metric-radio', 'value')
    ]
)
def update_channel_graph(selected_year, selected_channel, selected_thematics, selected_metric):
    filtered = df_chaine_transformed[
        (df_chaine_transformed['Année'] <= selected_year) &
        (df_chaine_transformed['Chaîne'] == selected_channel) &
        (df_chaine_transformed['Thématique'].isin(selected_thematics))
    ]

    if filtered.empty:
        return go.Figure()

    grouped = filtered.groupby(['Année', 'Thématique'])[selected_metric].sum().reset_index()

    fig = go.Figure()
    for thematic in selected_thematics:
        thematic_data = grouped[grouped['Thématique'] == thematic]
        fig.add_trace(go.Scatter(
            x=thematic_data['Année'],
            y=thematic_data[selected_metric],
            mode='lines+markers',
            name=thematic
        ))
    fig.update_layout(
        title=f"Évolution des thématiques pour {selected_channel} jusqu'à {selected_year}",
        xaxis_title="Année",
        yaxis_title="Valeur",
        legend_title="Thématique"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)



<IPython.core.display.Javascript object>

# Graphe Interactive avec votes et audio visuel

In [ ]:
import dash
from dash import dcc, html
import dash.dependencies as dd
import plotly.graph_objs as go

# Application Dash
app = dash.Dash(__name__)

# Années disponibles (communes aux deux datasets)
relevant_years = [2002, 2007, 2012, 2017, 2022, 2024]

# Chaînes disponibles
available_channels = df_chaine_transformed['Chaîne'].unique()

# Thématiques disponibles
available_thematics = df_chaine_transformed['Thématique'].unique()

# Sexes disponibles
available_sexes = df_all['Sexe'].unique()

app.layout = html.Div([
    # Dropdown pour choisir l'année
    html.Label("Choisir l'année"),
    dcc.Dropdown(
        id='year-dropdown',
        options=[{"label": str(y), "value": y} for y in relevant_years],
        value=2022,
        clearable=False
    ),

    # Radio boutons pour filtrer par sexe
    html.Label("Filtrer par sexe :"),
    dcc.Checklist(
        id='sex-checklist',
        options=[{"label": s, "value": s} for s in available_sexes],
        value=available_sexes,
        labelStyle={'display': 'inline-block'}
    ),

    # Graphique des élections
    dcc.Graph(id='election-graph'),

    # Liste déroulante pour choisir plusieurs chaînes télévisées
    html.Label("Choisir les chaînes télévisées :"),
    dcc.Dropdown(
        id='channel-dropdown',
        options=[{'label': c, 'value': c} for c in available_channels],
        value=[available_channels[0]],  # Chaîne par défaut
        multi=True
    ),

    # Checklist pour les thématiques
    html.Label("Choisir les thématiques :"),
    dcc.Checklist(
        id='thematic-checklist',
        options=[{'label': t, 'value': t} for t in available_thematics],
        value=[available_thematics[0]],  # Thématique par défaut
        labelStyle={'display': 'inline-block'}
    ),

    # Graphique des chaînes télévisées
    dcc.Graph(id='channel-graph'),

    # Bouton radio pour choisir entre Nb_Thème et Nb_temps
    html.Label("Choisir la métrique pour les chaînes télévisées :"),
    dcc.RadioItems(
        id='metric-radio',
        options=[
            {'label': 'Nombre de Thèmes', 'value': 'Nb_Thème'},
            {'label': 'Temps', 'value': 'Nb_temps'}
        ],
        value='Nb_Thème',  # Par défaut, on choisit le nombre de thèmes
        labelStyle={'display': 'inline-block'}
    )
])

# Callback pour le graphique des élections
@app.callback(
    dd.Output('election-graph', 'figure'),
    [
        dd.Input('year-dropdown', 'value'),
        dd.Input('sex-checklist', 'value')
    ]
)
def update_election_graph(selected_year, selected_sexes):
    filtered = df_all[(df_all['Année'] == selected_year) & (df_all['Sexe'].isin(selected_sexes))]
    grouped = filtered.groupby("Nuance").agg({"Voix": "sum", "Elu": "sum"}).reset_index()
    grouped = grouped.sort_values("Voix", ascending=False)

    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Voix"],
        name="Voix",
        marker_color='blue',
        text=grouped["Voix"],
        textposition='outside'
    ))
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Elu"],
        name="Élus",
        marker_color='orange',
        text=grouped["Elu"],
        textposition='outside'
    ))
    fig.update_layout(
        title=f"Élections {selected_year} : Voix et Élus par Nuance",
        xaxis_title="Nuance (Parti)",
        yaxis_title="Nombre total",
        barmode='group',
        bargap=0.15
    )
    return fig

# Callback pour le graphique des chaînes télévisées
@app.callback(
    dd.Output('channel-graph', 'figure'),
    [
        dd.Input('year-dropdown', 'value'),
        dd.Input('channel-dropdown', 'value'),
        dd.Input('thematic-checklist', 'value'),
        dd.Input('metric-radio', 'value')
    ]
)
def update_channel_graph(selected_year, selected_channels, selected_thematics, selected_metric):
    # Filtrer les données en fonction des sélections
    filtered = df_chaine_transformed[
        (df_chaine_transformed['Année'] <= selected_year) &
        (df_chaine_transformed['Chaîne'].isin(selected_channels)) &
        (df_chaine_transformed['Thématique'].isin(selected_thematics))
    ]

    if filtered.empty:
        return go.Figure()

    grouped = filtered.groupby(['Année', 'Thématique'])[selected_metric].sum().reset_index()

    fig = go.Figure()
    color_map = {
        thematic: f'rgba({i*40 % 255}, {i*60 % 255}, {i*80 % 255}, 0.7)'
        for i, thematic in enumerate(selected_thematics)
    }

    for thematic in selected_thematics:
        thematic_data = grouped[grouped['Thématique'] == thematic]
        fig.add_trace(go.Scatter(
            x=thematic_data['Année'],
            y=thematic_data[selected_metric],
            mode='lines+markers',
            name=thematic,
            line=dict(color=color_map[thematic])
        ))
    fig.update_layout(
        title=f"Évolution des thématiques pour les chaînes sélectionnées jusqu'à {selected_year}",
        xaxis_title="Année",
        yaxis_title="Valeur",
        legend_title="Thématique"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

# Graphe Interactive avec votes et nombre d'abonnée

In [ ]:
import dash
from dash import dcc, html
import dash.dependencies as dd
import plotly.graph_objs as go
import pandas as pd

# Chargement des données (df_all contient les données des élections)
df_twitter = pd.read_csv("compte_twiter_liées_audio_visuel_propre.csv", delimiter=";", encoding="utf-8")

# Prétraitement des catégories
df_twitter["Categorie"] = df_twitter["Categorie"].str.strip()
df_twitter["Année"] = pd.to_datetime(df_twitter["Date_Creation"], errors="coerce").dt.year
df_twitter_grouped = df_twitter.groupby(["Année", "Categorie"])["Nombre_Abonnes"].sum().reset_index()

# Années disponibles
available_years = sorted(df_all["Année"].unique())

# Catégories disponibles
available_categories = df_twitter_grouped["Categorie"].dropna().unique().tolist()

# Création de l'application Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    # Filtre pour l'année
    html.Div([
        html.Label("Choisir l'année", style={"font-weight": "bold"}),
        dcc.Dropdown(
            id="year-dropdown",
            options=[{"label": str(y), "value": y} for y in available_years],
            value=2022,  # Année par défaut
            clearable=False
        ),
    ], style={"margin-bottom": "20px"}),

    # Filtre pour les catégories
    html.Div([
        html.Label("Choisir une catégorie", style={"font-weight": "bold"}),
        dcc.RadioItems(
            id="category-radio",
            options=[{"label": c, "value": c} for c in available_categories],
            value=available_categories[0] if available_categories else None,  # Catégorie par défaut
            labelStyle={"display": "block"}
        ),
    ], style={"margin-bottom": "20px"}),

    # Graphique des élections
    html.Div([
        dcc.Graph(id="election-graph"),
    ], style={"margin-bottom": "20px"}),

    # Graphique des chaînes
    html.Div([
        dcc.Graph(id="twitter-graph"),
    ]),
])

@app.callback(
    dd.Output("election-graph", "figure"),
    dd.Input("year-dropdown", "value")
)
def update_election_graph(selected_year):
    # Filtrage des données pour l'année sélectionnée
    filtered_df = df_all[df_all["Année"] == selected_year]

    # Agrégation par nuance
    grouped = filtered_df.groupby("Nuance").agg({"Voix": "sum", "Elu": "sum"}).reset_index()

    # Création du graphique
    fig = go.Figure()
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Voix"],
        name="Voix",
        marker_color="blue",
        text=grouped["Voix"],
        textposition="outside"
    ))
    fig.add_trace(go.Bar(
        x=grouped["Nuance"],
        y=grouped["Elu"],
        name="Élus",
        marker_color="orange",
        text=grouped["Elu"],
        textposition="outside"
    ))
    fig.update_layout(
        title=f"Élections {selected_year} : Voix et Élus par Nuance",
        xaxis_title="Nuance (Parti)",
        yaxis_title="Nombre total",
        barmode="group",
        bargap=0.15
    )
    return fig

@app.callback(
    dd.Output("twitter-graph", "figure"),
    [dd.Input("year-dropdown", "value"),
     dd.Input("category-radio", "value")]
)
def update_twitter_graph(selected_year, selected_category):
    # Filtrage des données pour l'année et la catégorie sélectionnées
    filtered_df = df_twitter_grouped[(df_twitter_grouped["Année"] <= selected_year) &
                                     (df_twitter_grouped["Categorie"] == selected_category)]

    # Création du graphique
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=filtered_df["Année"],
        y=filtered_df["Nombre_Abonnes"],
        mode="lines+markers",
        name=selected_category,
        line=dict(width=2),
        marker=dict(size=8)
    ))
    fig.update_layout(
        title=f"Évolution des abonnés ({selected_category})",
        xaxis_title="Année",
        yaxis_title="Nombre d'abonnés",
        template="plotly_white"
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)


<ipython-input-11-4191682284b3>:12: UserWarning:

Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



<IPython.core.display.Javascript object>

In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from dash import dcc, html, Dash, Input, Output

# Charger les données
df_elections = df_all  # df_all, contenant 'Année', 'Nuance', 'Voix'
df_media = df_chaine

# Fusionner les deux jeux de données
df_combined = df_media.merge(df_elections, on="Année", how="inner")

# Normaliser les données
df_combined["Temps (%)"] = df_combined.groupby("Année")["Temps"].transform(lambda x: x / x.sum() * 100)
df_combined["Voix (%)"] = df_combined.groupby("Année")["Voix"].transform(lambda x: x / x.sum() * 100)

# Créer l'application Dash
app = Dash(__name__)

app.layout = html.Div([
    html.H1("Relations entre Thématiques Médiatiques et Résultats Électoraux"),

    # Dropdown pour choisir la thématique
    html.Label("Choisissez une thématique :"),
    dcc.Dropdown(
        id="theme-dropdown",
        options=[{"label": theme, "value": theme} for theme in df_combined["Thématique"].unique()],
        value=df_combined["Thématique"].unique()[0],
        clearable=False
    ),

    # Graphiques
    dcc.Graph(id="correlation-graph"),
    dcc.Graph(id="stacked-bar-graph"),
])

@app.callback(
    [Output("correlation-graph", "figure"),
     Output("stacked-bar-graph", "figure")],
    [Input("theme-dropdown", "value")]
)
def update_graphs(selected_theme):
    # Filtrer les données pour la thématique sélectionnée
    filtered = df_combined[df_combined["Thématique"] == selected_theme]

    if filtered.empty:
        # Si les données filtrées sont vides, renvoyer des graphiques vides
        return go.Figure(), go.Figure()

    # 1. Graphique de corrélation
    fig_corr = px.scatter(
        filtered,
        x="Temps (%)",
        y="Voix (%)",
        color="Nuance",
        title=f"Corrélation entre {selected_theme} et Résultats Électoraux",
        labels={"Temps (%)": "Temps Médiatique (%)", "Voix (%)": "Voix Électorales (%)"}
    )

    # 2. Graphique empilé
    fig_stacked = px.bar(
        filtered,
        x="Année",
        y="Temps (%)",
        color="Nuance",
        title=f"Évolution temporelle de {selected_theme} par orientation politique",
        barmode="stack"
    )

    return fig_corr, fig_stacked

if __name__ == "__main__":
    app.run_server(debug=True)


KeyError: 'Column not found: Temps'